# Coding Temple's Data Analytics Program
---
## Intro to API's Assignment
---
### Problem 1:
Check out this Pokemon API https://pokeapi.co/
Use the requests package to connect to this API and get and store data for 5 different pokemon.
Get the pokemons: name, atleast one ability's name, base_experience, and the URL for its sprite (an image that shows up on screen) for the 'front_shiny', attack base_state, hp base_stat, defense base_stat

**BONUS POINTS IF YOU CAN POPULATE THE IMAGE!**

In [41]:
import requests
import pandas as pd
from PIL import Image
from io import BytesIO 
from random import randint

In [50]:

my_team = ['Dragapult', 'Gallade', 'Dachsbun', 'Lucario', 'Coalossal', 'Sableye']

def try_shiny(poke):
    url = f'https://pokeapi.co/api/v2/pokemon/{poke.lower()}'
    response = requests.get(url).json()
    if response['sprites']['front_shiny']:
        return [True, response['sprites']['front_shiny']]
    else:
        return [False, response['sprites']['front_default']]

# def show_sprite(sprite_loc):
#     img_response = requests.get(sprite_loc)
#     return Image.open(BytesIO(img_response.content))


def pokemon_profile(pokes):
    team = []
    for poke in pokes:
        url = f'https://pokeapi.co/api/v2/pokemon/{poke.lower()}'
        response = requests.get(url).json()
        sprite_info = try_shiny(poke)
        # display(show_sprite(sprite_info[1]))
        poke_dict = {}
        poke_dict[poke] = {
            'species' : poke,
            'abilities': [response['abilities'][i]['ability']['name'] for i in range(len(response['abilities']))],
            'base exp': response['base_experience'],
            'attack (base)': response['stats'][1]['base_stat'],
            'defense (base)': response['stats'][2]['base_stat'],
            'hp (base)': response['stats'][0]['base_stat'],
            'has shiny': sprite_info[0],
            'shiny sprite': sprite_info[1]
        }
        team.append(poke_dict)
    return team

print(pokemon_profile(my_team))


[{'Dragapult': {'species': 'Dragapult', 'abilities': ['clear-body', 'infiltrator', 'cursed-body'], 'base exp': 300, 'attack (base)': 120, 'defense (base)': 75, 'hp (base)': 88, 'has shiny': True, 'shiny sprite': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/shiny/887.png'}}, {'Gallade': {'species': 'Gallade', 'abilities': ['steadfast', 'sharpness', 'justified'], 'base exp': 259, 'attack (base)': 125, 'defense (base)': 65, 'hp (base)': 68, 'has shiny': True, 'shiny sprite': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/shiny/475.png'}}, {'Dachsbun': {'species': 'Dachsbun', 'abilities': ['well-baked-body', 'aroma-veil'], 'base exp': None, 'attack (base)': 80, 'defense (base)': 115, 'hp (base)': 57, 'has shiny': False, 'shiny sprite': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/927.png'}}, {'Lucario': {'species': 'Lucario', 'abilities': ['steadfast', 'inner-focus', 'justified'], 'base exp': 184, 'attack 

### Problem 2:

I want to build the best Pokemon team I can and I need your help to do so! Using the API, build me a team of 6 pokemon who's base-stats are above 60. Return any additional information you think is needed to help understand why you chose that pokemon for the team!

In [48]:
def make_team():
    """
    Input: None required
    Output: Two lists (in a list)
    - List 1: Six Pokemon (each from any generation), with all stats at 60 or above.
    - List 2: Primary types of each Pokemon (secondary types not accounted for)
    Method:
    - Randomize a number between 1 and 1017 
    -- rejected pokemon will have their numbers added to a "rejected" list
    -- the random int is checked against the rejected list before querying API
    - Pull the json of that pokemon
    - look at its stat block, check that each stat is over 60
    - if the stats check out, see if the pokemon's primary type matches the primary type of an existing team member
    - If the type is new to the team:
    -- add pokemon's species to the team list
    -- add its type to have_type list
    -- add its pokedex number to rejected (the rejected list is the first line of defense against repeat pokemon)
    """
    team_list = []
    have_types = []
    rejected = []
    while len(team_list) < 6:
        valid_poke = True
        try_poke = randint(1,1017)
        if try_poke not in rejected:
            url = f'https://pokeapi.co/api/v2/pokemon/{try_poke}'
            trial_poke = requests.get(url).json()
            if trial_poke['types'][0]['type']['name'] in have_types:
                rejected.append(try_poke)
                valid_poke = False
            trial_statblock = trial_poke['stats']
            while valid_poke:
                for i in range(len(trial_statblock)):
                    if trial_statblock[i]['base_stat'] < 60:
                        rejected.append(try_poke)
                        valid_poke = False
                        break
                break
            if valid_poke == True:
                team_list.append(trial_poke['species']['name'].title())
                have_types.append(trial_poke['types'][0]['type']['name'])
                rejected.append(try_poke)
    return [team_list, have_types]

your_team = make_team()
print(your_team[0])
print(your_team[1])


['Roserade', 'Iron-Moth', 'Fezandipiti', 'Rayquaza', 'Kilowattrel', 'Iron-Jugulis']
['grass', 'fire', 'poison', 'dragon', 'electric', 'dark']


### Problem 3:

Create a function that does the following:
* Input: pokemon_name(user_defined) -> str
* Output: list object containing a dictionary of values from the response object.
* Please include the type, the most current raritey level, at least one held item, and all stats

In [ ]:
def poke_profile(a_poke):
    url = f'https://pokeapi.co/api/v2/pokemon/{a_poke.lower()}'
    response = requests.get(url).json()
    s_url = f'https://pokeapi.co/api/v2/pokemon-species/{a_poke.lower()}'
    s_response = requests.get(s_url).json()
    poke_deets = {}
    poke_deets['name'] = a_poke.title()
    poke_deets['types'] = [response['types'][i]['type']['name'] for i in range(len(response['types'])) ]
    poke_deets['capture_rate'] = s_response['capture_rate']
    poke_deets['held_items'] = [response['held_items'][i]['item']['name'] for i in range(len(response['held_items'])) ] 
    response_stat_block = response['stats']
    poke_deets['stats'] = {response_stat_block[i]['stat']['name'] : response_stat_block[i]['base_stat'] for i in range(len(response_stat_block))}
    return poke_deets
    
print(poke_profile('phantump'))



